In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st

electricity_data_to_load = "Resources/complete_dataset.csv"
electricity_data = pd.read_csv(electricity_data_to_load)
electricity_data.head(2)


,date,demand,RRP,demand_pos_RRP,RRP_positive,demand_neg_RRP,RRP_negative,frac_at_neg_RRP,min_temperature,max_temperature,solar_exposure,rainfall,school_day,holiday
0,2015-01-01,99635.03,25.633696,97319.240,26.415953,2315.790,-7.240000,0.020833,13.3,26.9,23.6,0.0,N,Y
1,2015-01-02,129606.01,33.138988,121082.015,38.837661,8523.995,-47.809777,0.062500,15.4,38.8,26.8,0.0,N,N


In [2]:
electricity_data_df = electricity_data.rename(columns={"date":"Date", "demand":"Demand (MWh)", "RRP": "RRP (AUD/MWh)",
                                                       "min_temperature":"Min Temperature", "max_temperature":"Max temperature",
                                                       "solar_exposure":"Total Sunlight Energy (MJ/m^2)", "rainfall": "Rainfall (mm)",
                                                      "school_day":"School Day", "holiday":"Holiday"})
electricity_data_df.head(2)

,Date,Demand (MWh),RRP (AUD/MWh),demand_pos_RRP,RRP_positive,demand_neg_RRP,RRP_negative,frac_at_neg_RRP,Min Temperature,Max temperature,Total Sunlight Energy (MJ/m^2),Rainfall (mm),School Day,Holiday
0,2015-01-01,99635.03,25.633696,97319.240,26.415953,2315.790,-7.240000,0.020833,13.3,26.9,23.6,0.0,N,Y
1,2015-01-02,129606.01,33.138988,121082.015,38.837661,8523.995,-47.809777,0.062500,15.4,38.8,26.8,0.0,N,N


In [10]:
electricity_data_clean = electricity_data_df.drop(electricity_data_df.columns[[3,4,5,6,7]],axis=1)

In [11]:
electricity_data_clean

,Date,Demand (MWh),RRP (AUD/MWh),Min Temperature,Max temperature,Total Sunlight Energy (MJ/m^2),Rainfall (mm),School Day,Holiday
0,2015-01-01,99635.030,25.633696,13.3,26.9,23.6,0.0,N,Y
1,2015-01-02,129606.010,33.138988,15.4,38.8,26.8,0.0,N,N
2,2015-01-03,142300.540,34.564855,20.0,38.2,26.5,0.0,N,N
3,2015-01-04,104330.715,25.005560,16.3,21.4,25.2,4.2,N,N
4,2015-01-05,118132.200,26.724176,15.0,22.0,30.7,0.0,N,N
...,...,...,...,...,...,...,...,...,...
2101,2020-10-02,99585.835,-6.076028,12.8,26.0,22.0,0.0,N,N
2102,2020-10-03,92277.025,-1.983471,17.4,29.4,19.8,0.0,N,N
2103,2020-10-04,94081.565,25.008614,13.5,29.5,8.4,0.0,N,N
2104,2020-10-05,113610.030,36.764701,9.1,12.7,7.3,12.8,N,N
